<h1><center>FDNY Data Processing<center></h1>

In this file I will be working with six sets of data: FDNY data of reported incidents (2 files from different sources), FDNY data of fire boxes locations (2 files from different sources), census tracts locations data, and NYC counties data. 

From the first two files I will filter out info about the gas leaks reported in NYC.<br> 
Using second pair of files I will find longitude and latitude of the incidents. <br>
Using the last two files I will find geoids of the incidents so then they can be grouped by the census tracts. This task will be completed using Carto.com. It is not automated, but takes much less time than using APIs. Census API will be used on the rows which Carto.com wasn't able to recognize.

In [144]:
import pandas as pd
import numpy as np
import os

In [145]:
dir_path = os.path.dirname(os.path.abspath("__file__"))

<h2><center> Data sources</center> </h2>

In [146]:
# Original FDNY data files won't be on GitHub since they are very large and require usage of cloud. 

# Original incidents data file1 can be found at https://data.cityofnewyork.us/Public-Safety/Incidents-Responded-to-by-Fire-Companies/tm6d-hbzd
# Original incidents data file2 can be found at https://data.cityofnewyork.us/Public-Safety/Fire-Incident-Dispatch-Data/8m42-w767

df_original1 = pd.read_csv(r'C:\Users\mskac\Desktop\Incidents_Responded_to_by_Fire_Companies.csv', low_memory=False)
df_original2 = pd.read_csv(r'C:\Users\mskac\Downloads\Fire_Incident_Dispatch_Data.csv', low_memory = False)

In [147]:
# the following files contain coordinates and long/lat of each fire box in NYC
# Location file 1: https://data.cityofnewyork.us/Public-Safety/In-Service-Alarm-Box-Locations/v57i-gtxb <br>
# Location file 2: http://www.poi-factory.com/node/11074

df_locations1 = pd.read_csv(dir_path + '\..\data\original\In_Service_Alarm_Box_Locations.csv', low_memory=False)
df_locations2 = pd.read_csv(dir_path + '\..\data\original\Fire_Boxes.csv', low_memory=False)

In [148]:
# the last file can be found on Carto.com if search for 'counties_ny' data in their datasets library

<h3>Filter gas leak incidents from original incidents data and save to files</h3>

In [149]:
# filter out only gas leaks data out of all FDNY calls (incidents file1)
df_incidents1 = df_original1[df_original1.INCIDENT_TYPE_DESC == "412 - Gas leak (natural gas or LPG)"]
df_incidents1.columns = map(str.lower, df_incidents1.columns)
df_incidents1.info()
df_incidents1.to_csv(r'C:\Users\mskac\machineLearning\GasLeakConEd\data\processed\Gas_Leak_Incidents_Responded_to_by_Fire_Companies.csv')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119324 entries, 3 to 2518753
Data columns (total 24 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   im_incident_key               119324 non-null  int64  
 1   fire_box                      119322 non-null  object 
 2   incident_type_desc            119324 non-null  object 
 3   incident_date_time            119324 non-null  object 
 4   arrival_date_time             118636 non-null  object 
 5   units_onscene                 118636 non-null  float64
 6   last_unit_cleared_date_time   119324 non-null  object 
 7   highest_level_desc            119322 non-null  object 
 8   total_incident_duration       119324 non-null  float64
 9   action_taken1_desc            119324 non-null  object 
 10  action_taken2_desc            115132 non-null  object 
 11  action_taken3_desc            113241 non-null  object 
 12  property_use_desc             119323 non-nu

In [150]:
df_original2.INCIDENT_CLASSIFICATION.unique()

array(['Medical - Assist Civilian', 'Alarm System - Defective',
       'Utility Emergency - Gas', 'Assist Civilian - Non-Medical',
       'Manhole Fire - Other', 'Non-Medical MFA - Phone',
       'Undefined Emergency', 'Demolition Debris or Rubbish Fire',
       'Medical - PD Link 10-91', 'Utility Emergency - Water',
       'Odor - Other Smoke', 'Odor - Other Than Smoke',
       'Vehicle Accident - Other',
       'Carbon Monoxide - Code 2 - Incident (1-9 ppm)',
       'Medical - EMS Link 10-91', 'Brush Fire',
       'Utility Emergency - Electric', 'Non-Medical MFA - ERS No Contact',
       "Multiple Dwelling 'A' - Food on the stove fire", 'Store Fire',
       'Defective Oil Burner', 'Medical MFA - PD Link',
       'Non-Medical MFA - Private Fire Alarm',
       'Carbon Monoxide - Code 1 - Investigation',
       'Alarm System - Testing', 'Other Transportation Fire',
       'Alarm System - Unnecessary', "Multiple Dwelling 'A' - Other fire",
       'Non-Medical 10-91 (Unnecessary Alarm)', 

In [151]:
# filter out only gas leaks data out of all FDNY calls (incidents file2)
df_incidents2 = df_original2[df_original2.INCIDENT_CLASSIFICATION == 'Utility Emergency - Gas']
df_incidents2.columns = map(str.lower, df_incidents2.columns)
df_incidents2.info()
df_incidents2.to_csv(r'C:\Users\mskac\machineLearning\GasLeakConEd\data\processed\Gas_Leak_Fire_Incident_Dispatch_Data.csv')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 156857 entries, 4 to 4079264
Data columns (total 29 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   starfire_incident_id            156857 non-null  float64
 1   incident_datetime               156857 non-null  object 
 2   alarm_box_borough               156857 non-null  object 
 3   alarm_box_number                156857 non-null  int64  
 4   alarm_box_location              156857 non-null  object 
 5   incident_borough                156857 non-null  object 
 6   zipcode                         156140 non-null  float64
 7   policeprecinct                  156141 non-null  float64
 8   citycouncildistrict             156055 non-null  float64
 9   communitydistrict               156141 non-null  float64
 10  communityschooldistrict         156137 non-null  float64
 11  congressionaldistrict           156055 non-null  float64
 12  alarm_source_de

<h3>What year do incident files cover? </h3>

In [152]:
df_incidents1.groupby(df_incidents1.incident_date_time.str[6:10]).count()

,im_incident_key,fire_box,incident_type_desc,incident_date_time,arrival_date_time,units_onscene,last_unit_cleared_date_time,highest_level_desc,total_incident_duration,action_taken1_desc,...,zip_code,borough_desc,floor,co_detector_present_desc,fire_origin_below_grade_flag,story_fire_origin_count,fire_spread_desc,detector_presence_desc,aes_presence_desc,standpipe_sys_present_flag
incident_date_time,,,,,,,,,,,,,,,,,,,,,
2013,14025,14025,14025,14025,13975,13975,14025,14025,14025,14025,...,14025,14025,7354,18,0,0,0,0,0,0
2014,21174,21173,21174,21174,21060,21060,21174,21174,21174,21174,...,21173,21174,12005,21,0,0,0,0,0,0
2015,25431,25430,25431,25431,25281,25281,25431,25430,25431,25431,...,25431,25431,14293,17,0,0,0,0,0,0
2016,23507,23507,23507,23507,23380,23380,23507,23506,23507,23507,...,23506,23507,13684,15,0,0,0,0,0,0
2017,24225,24225,24225,24225,24045,24045,24225,24225,24225,24225,...,24225,24225,14336,8,0,0,0,0,0,0
2018,10962,10962,10962,10962,10895,10895,10962,10962,10962,10962,...,10962,10962,6278,8,0,0,0,0,0,0


In [153]:
df_incidents2.groupby(df_incidents2.incident_datetime.str[6:10]).count()

,starfire_incident_id,incident_datetime,alarm_box_borough,alarm_box_number,alarm_box_location,incident_borough,zipcode,policeprecinct,citycouncildistrict,communitydistrict,...,first_activation_datetime,first_on_scene_datetime,incident_close_datetime,valid_dispatch_rspns_time_indc,valid_incident_rspns_time_indc,incident_response_seconds_qy,incident_travel_tm_seconds_qy,engines_assigned_quantity,ladders_assigned_quantity,other_units_assigned_quantity
incident_datetime,,,,,,,,,,,,,,,,,,,,,
2013,13945,13945,13945,13945,13945,13945,13883,13883,13872,13883,...,13944,13928,13945,13945,13945,13945,13945,13945,13945,13945
2014,20992,20992,20992,20992,20992,20992,20903,20903,20878,20903,...,20983,20953,20992,20992,20992,20992,20992,20992,20992,20992
2015,24975,24975,24975,24975,24975,24975,24866,24866,24834,24866,...,24971,24927,24975,24975,24975,24975,24975,24975,24975,24975
2016,23129,23129,23129,23129,23129,23129,23043,23043,23033,23043,...,23125,23091,23129,23129,23129,23129,23129,23129,23129,23129
2017,23969,23969,23969,23969,23969,23969,23877,23877,23873,23877,...,23960,23918,23969,23969,23969,23969,23969,23969,23969,23969
2018,22817,22817,22817,22817,22817,22817,22696,22696,22692,22696,...,22810,22760,22817,22817,22817,22817,22817,22817,22817,22817
2019,20312,20312,20312,20312,20312,20312,20186,20187,20187,20187,...,20301,20250,20312,20312,20312,20312,20312,20312,20312,20312
2020,6718,6718,6718,6718,6718,6718,6686,6686,6686,6686,...,6713,6695,6718,6718,6718,6718,6718,6718,6718,6718


<h3> Result of filtering incidents data</h3>

We had two files with incidents data from two different sources.

Filtering first file by incident type as gas leaks resulted in ~118000 rows.<br>
Filtering second file by incident type as gas leaks resulted in ~156000 rows.<br>

File 2 has information about reported incidents for 2013-2019, and part of 2020. The first file only has info about incidents for 2013-2017 and part of 2018. However, the first file has information about property use, which might be useful. So, we will analyze both of them.

<h2> <center>Prepare data for merging with fire box locations file</center></h2>

We will be merging by the 'fire_box' column.<br> 

Df_locations 'fire_box' column has the following format: capital letter for the borough (B-Brooklyn, X-Bronx, M-Manhattan, Q- Queens, R- Staten Island)+ 4 digits of fire_box ID'. Example: 'B0012'<br>
Df_incidents1 'fire_box' column only contains <= 4 digits of fire_box ID'. Example: 12<br>
Df_incidents2 'fire_box_number' column only contains <=digits of fire_box ID'. Example: 12<br>

Therefore, we need to make their formats the same

In [154]:
# get rid of unnecessary columns and drop nulls
df_incidents1.columns

Index(['im_incident_key', 'fire_box', 'incident_type_desc',
       'incident_date_time', 'arrival_date_time', 'units_onscene',
       'last_unit_cleared_date_time', 'highest_level_desc',
       'total_incident_duration', 'action_taken1_desc', 'action_taken2_desc',
       'action_taken3_desc', 'property_use_desc', 'street_highway', 'zip_code',
       'borough_desc', 'floor', 'co_detector_present_desc',
       'fire_origin_below_grade_flag', 'story_fire_origin_count',
       'fire_spread_desc', 'detector_presence_desc', 'aes_presence_desc',
       'standpipe_sys_present_flag'],
      dtype='object')

In [155]:
df_incidents1 = df_incidents1[['fire_box','incident_type_desc','incident_date_time','units_onscene','total_incident_duration','property_use_desc','borough_desc']]
df_incidents1 = df_incidents1.dropna()
df_incidents1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118633 entries, 3 to 2518734
Data columns (total 7 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   fire_box                 118633 non-null  object 
 1   incident_type_desc       118633 non-null  object 
 2   incident_date_time       118633 non-null  object 
 3   units_onscene            118633 non-null  float64
 4   total_incident_duration  118633 non-null  float64
 5   property_use_desc        118633 non-null  object 
 6   borough_desc             118633 non-null  object 
dtypes: float64(2), object(5)
memory usage: 7.2+ MB


In [156]:
df_incidents2.columns

Index(['starfire_incident_id', 'incident_datetime', 'alarm_box_borough',
       'alarm_box_number', 'alarm_box_location', 'incident_borough', 'zipcode',
       'policeprecinct', 'citycouncildistrict', 'communitydistrict',
       'communityschooldistrict', 'congressionaldistrict',
       'alarm_source_description_tx', 'alarm_level_index_description',
       'highest_alarm_level', 'incident_classification',
       'incident_classification_group', 'dispatch_response_seconds_qy',
       'first_assignment_datetime', 'first_activation_datetime',
       'first_on_scene_datetime', 'incident_close_datetime',
       'valid_dispatch_rspns_time_indc', 'valid_incident_rspns_time_indc',
       'incident_response_seconds_qy', 'incident_travel_tm_seconds_qy',
       'engines_assigned_quantity', 'ladders_assigned_quantity',
       'other_units_assigned_quantity'],
      dtype='object')

In [157]:
# get rid of unnecessary columns and drop nulls
df_incidents2 = df_incidents2[['alarm_box_number','incident_classification','incident_datetime','incident_close_datetime','engines_assigned_quantity','incident_borough']]
df_incidents2 = df_incidents2.dropna()
df_incidents2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 156857 entries, 4 to 4079264
Data columns (total 6 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   alarm_box_number           156857 non-null  int64 
 1   incident_classification    156857 non-null  object
 2   incident_datetime          156857 non-null  object
 3   incident_close_datetime    156857 non-null  object
 4   engines_assigned_quantity  156857 non-null  int64 
 5   incident_borough           156857 non-null  object
dtypes: int64(2), object(4)
memory usage: 8.4+ MB


In [158]:
# rename columns so they have same names in both dataframes 1 and 2

df_incidents2 = df_incidents2.rename(columns = {'alarm_box_number': 'fire_box', 'incident_classification':'incident_type_desc', 'incident_datetime': 'incident_date_time', 'incident_borough':'borough_desc'})

In [166]:
df_incidents1['fire_box'] = df_incidents1['fire_box'].str.zfill(4)
df_incidents2['fire_box'] = df_incidents2['fire_box'].astype('str').str.zfill(4)

In [171]:
df_incidents2.head()

,fire_box,incident_type_desc,incident_date_time,incident_close_datetime,engines_assigned_quantity,borough_desc,boro_letter
4,7412,Utility Emergency - Gas,01/01/2013 12:02:32 AM,01/01/2013 12:40:11 AM,2,QUEENS,R
197,4504,Utility Emergency - Gas,01/01/2013 01:53:19 AM,01/01/2013 02:18:14 AM,2,QUEENS,R
218,3064,Utility Emergency - Gas,01/01/2013 02:07:29 AM,01/01/2013 02:22:01 AM,2,BRONX,R
292,3652,Utility Emergency - Gas,01/01/2013 02:47:57 AM,01/01/2013 03:10:23 AM,3,BROOKLYN,R
467,2170,Utility Emergency - Gas,01/01/2013 04:53:39 AM,01/01/2013 05:15:52 AM,2,BRONX,R


In [172]:
df_incidents1['boro_letter'] = df_incidents1['borough_desc'].apply(lambda x: 'B' if x[0] == '4' else 'M' if x[0] == '1' else 'X' if x[0] == '2' else 'Q' if x[0]=='5' else 'R')
df_incidents2['boro_letter'] = df_incidents2['borough_desc'].apply(lambda x: 'B' if x == 'BROOKLYN' else 'M' if x == 'MANHATTAN' else 'X' if x == 'BRONX' else 'Q' if x=='QUEENS' else 'R')

In [173]:
#make sure we didn't change any data
df_incidents1.groupby('boro_letter').count()


,fire_box,incident_type_desc,incident_date_time,units_onscene,total_incident_duration,property_use_desc,borough_desc
boro_letter,,,,,,,
B,33292,33292,33292,33292,33292,33292,33292
M,32174,32174,32174,32174,32174,32174,32174
Q,22251,22251,22251,22251,22251,22251,22251
R,5349,5349,5349,5349,5349,5349,5349
X,25567,25567,25567,25567,25567,25567,25567


In [174]:
df_incidents1.groupby('borough_desc').count()

,fire_box,incident_type_desc,incident_date_time,units_onscene,total_incident_duration,property_use_desc,boro_letter
borough_desc,,,,,,,
1 - Manhattan,32174,32174,32174,32174,32174,32174,32174
2 - Bronx,25567,25567,25567,25567,25567,25567,25567
3 - Staten Island,5349,5349,5349,5349,5349,5349,5349
4 - Brooklyn,33292,33292,33292,33292,33292,33292,33292
5 - Queens,22251,22251,22251,22251,22251,22251,22251


In [175]:
#make sure we didn't change any data
df_incidents2.groupby('boro_letter').count()


,fire_box,incident_type_desc,incident_date_time,incident_close_datetime,engines_assigned_quantity,borough_desc
boro_letter,,,,,,
B,43711,43711,43711,43711,43711,43711
M,43015,43015,43015,43015,43015,43015
Q,28381,28381,28381,28381,28381,28381
R,6906,6906,6906,6906,6906,6906
X,34844,34844,34844,34844,34844,34844


In [176]:
df_incidents2.groupby('borough_desc').count()

,fire_box,incident_type_desc,incident_date_time,incident_close_datetime,engines_assigned_quantity,boro_letter
borough_desc,,,,,,
BRONX,34844,34844,34844,34844,34844,34844
BROOKLYN,43711,43711,43711,43711,43711,43711
MANHATTAN,43015,43015,43015,43015,43015,43015
QUEENS,28381,28381,28381,28381,28381,28381
RICHMOND / STATEN ISLAND,6906,6906,6906,6906,6906,6906


In [177]:
df_incidents1['fire_box'] = df_incidents1.boro_letter + df_incidents1.fire_box
df_incidents2['fire_box'] = df_incidents2.boro_letter + df_incidents2.fire_box

In [180]:
df_incidents1.head()

,fire_box,incident_type_desc,incident_date_time,units_onscene,total_incident_duration,property_use_desc,borough_desc,boro_letter
3,Q7412,412 - Gas leak (natural gas or LPG),01/01/2013 12:02:32 AM,4.0,2259.0,429 - Multifamily dwelling,5 - Queens,Q
159,Q4504,412 - Gas leak (natural gas or LPG),01/01/2013 01:53:19 AM,5.0,1495.0,"962 - Residential street, road or residential ...",5 - Queens,Q
175,X3064,412 - Gas leak (natural gas or LPG),01/01/2013 02:07:29 AM,5.0,872.0,419 - 1 or 2 family dwelling,2 - Bronx,X
226,B3652,412 - Gas leak (natural gas or LPG),01/01/2013 02:47:57 AM,6.0,1346.0,419 - 1 or 2 family dwelling,4 - Brooklyn,B
260,Q6180,412 - Gas leak (natural gas or LPG),01/01/2013 03:10:53 AM,6.0,1498.0,"962 - Residential street, road or residential ...",5 - Queens,Q


In [181]:
df_incidents2.head()

,fire_box,incident_type_desc,incident_date_time,incident_close_datetime,engines_assigned_quantity,borough_desc,boro_letter
4,Q7412,Utility Emergency - Gas,01/01/2013 12:02:32 AM,01/01/2013 12:40:11 AM,2,QUEENS,Q
197,Q4504,Utility Emergency - Gas,01/01/2013 01:53:19 AM,01/01/2013 02:18:14 AM,2,QUEENS,Q
218,X3064,Utility Emergency - Gas,01/01/2013 02:07:29 AM,01/01/2013 02:22:01 AM,2,BRONX,X
292,B3652,Utility Emergency - Gas,01/01/2013 02:47:57 AM,01/01/2013 03:10:23 AM,3,BROOKLYN,B
467,X2170,Utility Emergency - Gas,01/01/2013 04:53:39 AM,01/01/2013 05:15:52 AM,2,BRONX,X


In [ ]:
df_incidents1.drop(['boro_letter', 'incident_type_desc'], axis = 1, inplace = True)

In [186]:
df_incidents2['total_incident_duration'] = pd.to_datetime(df_incidents2.incident_close_datetime) - pd.to_datetime(df_incidents2.incident_date_time)
df_incidents2.drop(['boro_letter', 'incident_type_desc', 'incident_close_datetime'], axis = 1, inplace = True)

In [198]:
df_incidents2['total_incident_duration'] = df_incidents2['total_incident_duration'].dt.total_seconds()
df_incidents2.head()

,fire_box,incident_date_time,engines_assigned_quantity,borough_desc,total_incident_duration
4,Q7412,01/01/2013 12:02:32 AM,2,QUEENS,2259.0
197,Q4504,01/01/2013 01:53:19 AM,2,QUEENS,1495.0
218,X3064,01/01/2013 02:07:29 AM,2,BRONX,872.0
292,B3652,01/01/2013 02:47:57 AM,3,BROOKLYN,1346.0
467,X2170,01/01/2013 04:53:39 AM,2,BRONX,1333.0


<center><h2>Merge data with the fire boxes locations file to find long and lat</h2></center>

<h3> Locations file 1 and Incidents file 1</h3>

In [201]:
df_locations1.columns

Index(['BOROBOX', 'BOX_TYPE', 'LOCATION', 'ZIP', 'BOROUGH', 'COMMUNITYDISTICT',
       'CITYCOUNCIL', 'LATITUDE', 'LONGITUDE', 'Location Point', 'Zip Codes',
       'Community Districts', 'Borough Boundaries', 'City Council Districts',
       'Police Precincts'],
      dtype='object')

In [202]:
df_locations1 = df_locations1[['BOROBOX', 'LATITUDE', 'LONGITUDE', 'Police Precincts']]
df_locations1.columns = map(str.lower, df_locations1.columns)

In [204]:
df_locations1['fire_box'] = df_locations1['borobox']

In [205]:
df_merged = pd.merge(df_incidents1, df_locations1, how='inner', on = 'fire_box')
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97725 entries, 0 to 97724
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   fire_box                 97725 non-null  object 
 1   incident_date_time       97725 non-null  object 
 2   units_onscene            97725 non-null  float64
 3   total_incident_duration  97725 non-null  float64
 4   property_use_desc        97725 non-null  object 
 5   borough_desc             97725 non-null  object 
 6   borobox                  97725 non-null  object 
 7   latitude                 97725 non-null  float64
 8   longitude                97725 non-null  float64
 9   police precincts         97721 non-null  float64
dtypes: float64(5), object(5)
memory usage: 8.2+ MB


<h3> Locations file 2 and Incidents file 1</h3>

In [206]:
df_locations2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16284 entries, 0 to 16283
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   longitude  16284 non-null  float64
 1   latitude   16284 non-null  float64
 2   fire_box   16284 non-null  object 
 3   address    16284 non-null  object 
dtypes: float64(2), object(2)
memory usage: 509.0+ KB


In [207]:
df_merged2 = pd.merge(df_incidents1, df_locations2, how='inner', on = 'fire_box')
df_merged2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114994 entries, 0 to 114993
Data columns (total 9 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   fire_box                 114994 non-null  object 
 1   incident_date_time       114994 non-null  object 
 2   units_onscene            114994 non-null  float64
 3   total_incident_duration  114994 non-null  float64
 4   property_use_desc        114994 non-null  object 
 5   borough_desc             114994 non-null  object 
 6   longitude                114994 non-null  float64
 7   latitude                 114994 non-null  float64
 8   address                  114994 non-null  object 
dtypes: float64(4), object(5)
memory usage: 8.8+ MB


<h3> Results of the merging</h3>

We had two files with locations of fire boxes from two different sources.

Merging of incidents data with the file 1 resulted in loosing 20908 rows (17%).<br>
Merging of incidents data with the file 2 resulted in loosing 3639 rows (3%).<br>

I chose to use file 2 since it gave more information and saved more rows in the incidents data.

In [208]:
df_merged_2013_2018 = df_merged2

In [209]:
df_merged_2013_2018.head()

,fire_box,incident_date_time,units_onscene,total_incident_duration,property_use_desc,borough_desc,longitude,latitude,address
0,Q7412,01/01/2013 12:02:32 AM,4.0,2259.0,429 - Multifamily dwelling,5 - Queens,-73.91579,40.75754,Broadway && 44th St
1,Q7412,04/20/2013 11:43:06 AM,3.0,872.0,429 - Multifamily dwelling,5 - Queens,-73.91579,40.75754,Broadway && 44th St
2,Q7412,08/13/2013 06:03:48 AM,3.0,1118.0,429 - Multifamily dwelling,5 - Queens,-73.91579,40.75754,Broadway && 44th St
3,Q7412,01/03/2014 12:04:59 PM,5.0,1374.0,429 - Multifamily dwelling,5 - Queens,-73.91579,40.75754,Broadway && 44th St
4,Q7412,03/12/2014 04:19:44 PM,4.0,4617.0,"962 - Residential street, road or residential ...",5 - Queens,-73.91579,40.75754,Broadway && 44th St


In [210]:
df_merged_2013_2020 = pd.merge(df_incidents2, df_locations2, how='inner', on = 'fire_box')
df_merged_2013_2020.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 152148 entries, 0 to 152147
Data columns (total 8 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   fire_box                   152148 non-null  object 
 1   incident_date_time         152148 non-null  object 
 2   engines_assigned_quantity  152148 non-null  int64  
 3   borough_desc               152148 non-null  object 
 4   total_incident_duration    152148 non-null  float64
 5   longitude                  152148 non-null  float64
 6   latitude                   152148 non-null  float64
 7   address                    152148 non-null  object 
dtypes: float64(3), int64(1), object(4)
memory usage: 10.4+ MB


In [211]:
df_merged_2013_2020.head()

,fire_box,incident_date_time,engines_assigned_quantity,borough_desc,total_incident_duration,longitude,latitude,address
0,Q7412,01/01/2013 12:02:32 AM,2,QUEENS,2259.0,-73.91579,40.75754,Broadway && 44th St
1,Q7412,04/20/2013 11:43:06 AM,2,QUEENS,872.0,-73.91579,40.75754,Broadway && 44th St
2,Q7412,08/13/2013 06:03:48 AM,2,QUEENS,1118.0,-73.91579,40.75754,Broadway && 44th St
3,Q7412,01/03/2014 12:04:59 PM,2,QUEENS,1374.0,-73.91579,40.75754,Broadway && 44th St
4,Q7412,03/12/2014 04:19:44 PM,2,QUEENS,4617.0,-73.91579,40.75754,Broadway && 44th St


In [213]:
df_merged_2013_2018.to_csv(dir_path + '\..\data\processed\FDNY_2013_2018_with_property_use.csv')
df_merged_2013_2020.to_csv(dir_path + '\..\data\processed\FDNY_2013_2020_no_property_use.csv')

<h2> <center> Find census tract for each incident </center> </h2>

Using Carto.com site I will find geoids of each incident. <br>
1. Download geojson file with boundaries of 2010 census tract from https://data.cityofnewyork.us/City-Government/2010-Census-Tracts/fxpq-c8ku <br>
2. On carto.com find dataset 'ny_counties'
3. Upload FDNY_2013_2018 and FDNY_2013_2020, Census Tracts files to Carto.com
4. On FDNY files run this query:
SELECT x.*,y.*
FROM 
  (SELECT a.*,b.fips 
  FROM fdny_2013_2018_with_property_use as a,counties_ny as b
  WHERE ST_Within(a.the_geom,b.the_geom)) as x,
table_2010_census_tracts as y
WHERE ST_Within(x.the_geom, y.the_geom)<br> 
5. Export both files and save to dir_path + '\..\data\processed\FDNY_2013_2018_with_property_use_GEOID.csv', 
dir_path + '\..\data\processed\FDNY_2013_2020_no_property_use_GEOID.csv'
6. 

In [328]:
df_incidents_carto2013_2020 =  pd.read_csv(dir_path + '\..\data\processed\FDNY_2013_2020_no_property_use_GEOID.csv')

In [236]:
df_notrecognized = df_merged_2013_2020[~df_merged_2013_2020['fire_box'].isin(df_incidents_carto2013_2020['fire_box'])]

In [252]:
df_notrecognized['block_fips'] = 'temp'
df_notrecognized.head()

C:\Users\mskac\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,fire_box,incident_date_time,engines_assigned_quantity,borough_desc,total_incident_duration,longitude,latitude,address,block_fips
113,B3652,01/01/2013 02:47:57 AM,3,BROOKLYN,1346.0,-73.96216,40.57961,Oceanview Ave & Brighton 6th St,temp
114,B3652,04/27/2013 02:16:59 PM,2,BROOKLYN,1245.0,-73.96216,40.57961,Oceanview Ave & Brighton 6th St,temp
115,B3652,08/31/2013 11:51:39 PM,3,BROOKLYN,859.0,-73.96216,40.57961,Oceanview Ave & Brighton 6th St,temp
116,B3652,11/08/2013 03:29:47 PM,4,BROOKLYN,833.0,-73.96216,40.57961,Oceanview Ave & Brighton 6th St,temp
117,B3652,03/23/2014 08:23:41 AM,2,BROOKLYN,932.0,-73.96216,40.57961,Oceanview Ave & Brighton 6th St,temp


In [297]:
df_incidents_carto2013_2020.head()

,the_geom,cartodb_id,field_1,fire_box,incident_date_time,engines_assigned_quantity,borough_desc,total_incident_duration,longitude,latitude,address,fips,county,ntaname,boro_name,ct2010,shape_area
0,0101000020E6100000276BD443347E52C0789CA223B94C...,9123,9122,B4235,01/06/2013 07:05:52 PM,2,BROOKLYN,1035,-73.97194,40.59940,Avenue T & West St,36047,Kings,Bensonhurst East,Brooklyn,40000,2.058373e+06
1,0101000020E6100000F1BA7EC16E7A52C0F27B9BFEEC4F...,17200,17199,B3183,01/13/2013 07:21:51 AM,3,BROOKLYN,1469,-73.91301,40.62442,Avenue M & E 70th St,36047,Kings,Georgetown-Marine Park-Bergen Beach-Mill Basin,Brooklyn,69602,7.888410e+06
2,0101000020E6100000C7EF6DFAB37D52C01ADD41EC4C51...,20280,20279,B2475,07/30/2017 04:40:45 PM,2,BROOKLYN,481,-73.96411,40.63516,Newkirk Ave & Rugby Rd,36047,Kings,Flatbush,Brooklyn,52000,2.056594e+06
3,0101000020E61000005DFE43FAED7B52C0D908C4EBFA55...,30406,30405,B1012,02/24/2020 10:11:19 AM,2,BROOKLYN,558,-73.93640,40.67172,Troy Ave & Sterling Pl,36047,Kings,Crown Heights North,Brooklyn,35300,1.755456e+06
4,0101000020E61000009B20EA3E007F52C08F19A88C7F57...,34816,34815,B0582,12/19/2019 10:18:32 AM,2,BROOKLYN,1158,-73.98439,40.68358,Nevins St & Wyckoff St,36047,Kings,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,Brooklyn,7100,1.840259e+06


<h2> Get the missing data (some locations were not in the ny_county data)</h2>

Use Census API to get fips codes for the missing fire boxes

In [262]:
pd.options.mode.chained_assignment = None

In [275]:
import urllib, json, requests

#this will take ~2 hours to run
for index, row in df_notrecognized.iterrows():
    with urllib.request.urlopen("https://geo.fcc.gov/api/census/area?lat={0}&lon={1}&format=json".format(row['latitude'], row['longitude'])) as url:
        data = json.loads(url.read().decode())
        df_notrecognized.loc[index, 'block_fips'] = data['results'][0]['block_fips']

In [276]:
df_notrecognized.to_csv(dir_path + '\..\data\processed\FDNY_2013_2020_no_property_use_GEOID_notrecogn.csv')

In [329]:
df_incidents_carto2013_2018 =  pd.read_csv(dir_path + '\..\data\processed\FDNY_2013_2018_with_property_use_GEOID.csv')
df_notrecognized2 = df_merged_2013_2018[~df_merged_2013_2018['fire_box'].isin(df_incidents_carto2013_2018['fire_box'])]

In [277]:
#this will take ~2 hours to run
for index, row in df_notrecognized2.iterrows():
    with urllib.request.urlopen("https://geo.fcc.gov/api/census/area?lat={0}&lon={1}&format=json".format(row['latitude'], row['longitude'])) as url:
        data = json.loads(url.read().decode())
        df_notrecognized2.loc[index, 'block_fips'] = data['results'][0]['block_fips']
        
df_notrecognized2.to_csv(dir_path + '\..\data\processed\FDNY_2013_2018_with_property_use_GEOID_notrecogn.csv')

<h3>Concatinate the missing data to the main data</h3>

In [330]:
df_incidents_carto2013_2018['ct2010'] = df_incidents_carto2013_2018['ct2010'].astype('str').str.zfill(6)
df_incidents_carto2013_2018['geoid'] = df_incidents_carto2013_2018['fips'].astype(str)+df_incidents_carto2013_2018['ct2010'].astype(str)
df_incidents_carto2013_2018 = df_incidents_carto2013_2018[['geoid', 'fire_box', 'property_use_desc','ntaname','boro_name','incident_date_time','units_onscene','total_incident_duration']]

In [331]:
df_incidents_carto2013_2020['ct2010'] = df_incidents_carto2013_2020['ct2010'].astype('str').str.zfill(6)
df_incidents_carto2013_2020['geoid'] = df_incidents_carto2013_2020['fips'].astype(str)+df_incidents_carto2013_2020['ct2010'].astype(str)
df_incidents_carto2013_2020 = df_incidents_carto2013_2020[['geoid', 'fire_box', 'ntaname','boro_name','incident_date_time','engines_assigned_quantity','total_incident_duration']]

In [332]:
df_incidents_carto2013_2020.head()

,geoid,fire_box,ntaname,boro_name,incident_date_time,engines_assigned_quantity,total_incident_duration
0,36047040000,B4235,Bensonhurst East,Brooklyn,01/06/2013 07:05:52 PM,2,1035
1,36047069602,B3183,Georgetown-Marine Park-Bergen Beach-Mill Basin,Brooklyn,01/13/2013 07:21:51 AM,3,1469
2,36047052000,B2475,Flatbush,Brooklyn,07/30/2017 04:40:45 PM,2,481
3,36047035300,B1012,Crown Heights North,Brooklyn,02/24/2020 10:11:19 AM,2,558
4,36047007100,B0582,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,Brooklyn,12/19/2019 10:18:32 AM,2,1158


In [312]:
df_incidents_carto2013_2018.head()

,geoid,fire_box,property_use_desc,ntaname,boro_name,incident_date_time,units_onscene,total_incident_duration,shape_area
0,36047039200,B3361,429 - Multifamily dwelling,Homecrest,Brooklyn,11/08/2015 11:24:36 PM,5,1055,1.875666e+06
1,36047018400,B2821,"960 - Street, other",Bensonhurst West,Brooklyn,11/30/2015 12:11:23 PM,3,2003,1.609240e+06
2,36047107800,B1966,429 - Multifamily dwelling,East New York,Brooklyn,10/01/2013 10:04:59 AM,5,700,5.449630e+06
3,36047089200,B1635,429 - Multifamily dwelling,Brownsville,Brooklyn,05/29/2014 07:28:14 PM,6,1534,1.932413e+06
4,36047055200,B4269,429 - Multifamily dwelling,Madison,Brooklyn,03/11/2013 05:36:33 PM,3,897,1.805017e+06


In [320]:
concat1 = pd.read_csv(r'..\data\processed\fdny_2013_2018_with_property_use_geoid_notrecogn.csv')
concat2 = pd.read_csv(r'..\data\processed\fdny_2013_2020_no_property_use_geoid_notrecogn.csv')

In [322]:
concat1['geoid'] = concat1['block_fips'].astype(str).str[:11]
concat1.drop(['block_fips'], axis = 1, inplace = True)

concat2['geoid'] = concat2['block_fips'].astype(str).str[:11]
concat2.drop(['block_fips'], axis = 1, inplace = True)

In [325]:
concat1 = concat1[['geoid', 'fire_box', 'property_use_desc','ntaname','boroname','incident_date_time','units_onscene','total_incident_duration']]
concat2 = concat2[['geoid', 'fire_box', 'ntaname','boroname','incident_date_time','engines_assigned_quantity','total_incident_duration']]

In [333]:
df_incidents_carto2013_2018.rename(columns={'boro_name': 'boroname'}, inplace = True)
df_incidents_carto2013_2020.rename(columns={'boro_name': 'boroname'}, inplace = True)

In [336]:
df_full_2013_2018 = pd.concat([df_incidents_carto2013_2018, concat1], ignore_index=True)
df_full_2013_2020 = pd.concat([df_incidents_carto2013_2020, concat2], ignore_index=True)

In [338]:
df_full_2013_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152136 entries, 0 to 152135
Data columns (total 7 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   geoid                      152136 non-null  object
 1   fire_box                   152136 non-null  object
 2   ntaname                    152136 non-null  object
 3   boroname                   152136 non-null  object
 4   incident_date_time         152136 non-null  object
 5   engines_assigned_quantity  152136 non-null  int64 
 6   total_incident_duration    152136 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 8.1+ MB


In [340]:
df_full_2013_2018.to_csv(dir_path + '\..\data\processed\FULL_fdny_2013_2018.csv')
df_full_2013_2020.to_csv(dir_path + '\..\data\processed\FULL_fdny_2013_2020.csv')

In [341]:
df_full_2013_2018.head()

,geoid,fire_box,property_use_desc,ntaname,boroname,incident_date_time,units_onscene,total_incident_duration
0,36047039200,B3361,429 - Multifamily dwelling,Homecrest,Brooklyn,11/08/2015 11:24:36 PM,5,1055
1,36047018400,B2821,"960 - Street, other",Bensonhurst West,Brooklyn,11/30/2015 12:11:23 PM,3,2003
2,36047107800,B1966,429 - Multifamily dwelling,East New York,Brooklyn,10/01/2013 10:04:59 AM,5,700
3,36047089200,B1635,429 - Multifamily dwelling,Brownsville,Brooklyn,05/29/2014 07:28:14 PM,6,1534
4,36047055200,B4269,429 - Multifamily dwelling,Madison,Brooklyn,03/11/2013 05:36:33 PM,3,897
